In [1]:
from rolling_zarr_array_store import OffsetArrayStoreMixin
import numpy as np
import zarr
import json
import datetime

In [2]:
class Store(OffsetArrayStoreMixin, dict):
    pass

In [3]:
store = Store(cache_offset=False)

## Day 0
Create an array of 'the next 5 days'

In [4]:
dt_format="%Y-%m-%d"
next_5_days = [(datetime.datetime.now().date() + datetime.timedelta(days=i)).strftime(dt_format) for i in range(5)]
data = zarr.array(np.array(next_5_days), chunks=(1,), store=store)
data[:]

array(['2019-03-14', '2019-03-15', '2019-03-16', '2019-03-17',
       '2019-03-18'], dtype='<U10')

# Subsequent days...

By moving the offset forward by one and then appending a new date to the end we roll the zarr forward in time

In [5]:
def progress_one_day(data):
    current_end_date = datetime.datetime.strptime(str(data[-1]), "%Y-%m-%d").date()
    next_end_date = current_end_date + datetime.timedelta(days=1)
    
    current_offset = json.loads(store.get('.zattrs','{}')).get('_offset', [0])
    next_offset = [current_offset[0] +1]
    
    # move forward the offset then prepend the data
    data.store['.zattrs'] = json.dumps({'_offset': next_offset})
    data[-1] = next_end_date.strftime(dt_format)
    
    

In [6]:
for i in range(1, 6):
    
    progress_one_day(data)
    data = zarr.open(store)
    print(f"Day {i}:{data[:]}")

Day 1:['2019-03-15' '2019-03-16' '2019-03-17' '2019-03-18' '2019-03-19']
Day 2:['2019-03-16' '2019-03-17' '2019-03-18' '2019-03-19' '2019-03-20']
Day 3:['2019-03-17' '2019-03-18' '2019-03-19' '2019-03-20' '2019-03-21']
Day 4:['2019-03-18' '2019-03-19' '2019-03-20' '2019-03-21' '2019-03-22']
Day 5:['2019-03-19' '2019-03-20' '2019-03-21' '2019-03-22' '2019-03-23']
